# NLP - Sentiment Analysis avec Bag Of Words

## Objectif
Classifier des tweets pour déterminer s'ils sont positifs ou négatifs en utilisant l'approche **Bag of Words et un modèle de régression logistique**.

## Etapes

- **Filtrage des Tweets**, Ratio des Tweets Positifs et Négatifs
- **Train_Test_Split** - Séparation des données
- Entrainement du modèle **CountVectorizer**
- **Régression Logistique**
- ***Bonus*** : afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs)  
- Entrainement avec **TfidfVectorizer**
- Comparatif des modèles CountVectorizer et TfidfVectorizer

## Étape 1 : Importer les bibliothèques nécessaires

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


---

## Étape 2 : Charger et explorer les données

### Chargement des Tweets

In [3]:
# Charger les données depuis l'URL
url = "https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"
df = pd.read_csv(url, compression='zip')

# Aperçu des données
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27480 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27480 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


### Filtrer les tweets positifs et négatifs

In [5]:
df['sentiment'].value_counts()

sentiment
neutral     11117
positive     8582
negative     7781
Name: count, dtype: int64

In [6]:
# Filtrer des tweets
df = df[df['sentiment'].isin(['positive', 'negative'])]

# Proportions de chaque classe
print(df['sentiment'].value_counts(normalize=True) * 100)

sentiment
positive    52.447595
negative    47.552405
Name: proportion, dtype: float64


**Ratio des Tweets** :
- ***Tweets positifs*** : 52,44%
- ***Tweets négatifs*** : 47,55%

---

## Étape 3 : Préparer les données pour l'entraînement

### Séparer les ***caractéritisques*** (Tweets) et les ***labels*** (Sentiment)

In [7]:
X = df['text']
y = df['sentiment']

# Diviser les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

---

## Étape 4 : Vectorisation avec CountVectorizer

### Créer et ajuster un vecteur de mots

In [8]:
# Initialisation du CountVectorizer
vectorizer = CountVectorizer()

# Appliquer le vectorizer sur les données d'entraînement
X_train_CV = vectorizer.fit_transform(X_train)

# Transformer les données de test sans ré-entraîner
X_test_CV = vectorizer.transform(X_test)

# Vérifier la forme des matrices
print(f"Dimensions X_train_CV : {X_train_CV.shape}")
print(f"Dimensions X_test_CV : {X_test_CV.shape}")

Dimensions X_train_CV : (12272, 15806)
Dimensions X_test_CV : (4091, 15806)


---

## Étape 5 : Entraîner le modèle

### Créer et entraîner un modèle de régression logistique

In [9]:
# Initialisation du modèle
model = LogisticRegression()

# Entraîner le modèle
model.fit(X_train_CV, y_train)

LogisticRegression()

---

## Étape 6 : Évaluation des performances

### Évaluer la performance sur les ensembles d'entraînement et de test

In [10]:
# Prédictions sur les ensembles
train_accuracy = model.score(X_train_CV, y_train)
test_accuracy = model.score(X_test_CV, y_test)

print(f"Train Accuracy: {train_accuracy:.3f}")
print(f"Test Accuracy: {test_accuracy:.3f}")

# Rapport de classification complet
y_pred = model.predict(X_test_CV)
print(classification_report(y_test, y_pred))

Train Accuracy: 0.966
Test Accuracy: 0.877
              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      1935
    positive       0.89      0.88      0.88      2156

    accuracy                           0.88      4091
   macro avg       0.88      0.88      0.88      4091
weighted avg       0.88      0.88      0.88      4091



---

## Étape 7 : *Bonus* - Analyser les prédictions incorrectes

### Identifier et examiner les erreurs

In [11]:
# Identifier les faux positifs et faux négatifs
mal_prédit = X_test[y_test != y_pred]

# Afficher un échantillon de 10 erreurs
print("Echantillon de 10 prédictions incorrectes :\n")
for text, true_label, pred_label in zip(mal_prédit[:10], y_test[y_test != y_pred], y_pred[y_test != y_pred]):
    print(f"Tweet: {text}\nLabel réel: {true_label}, Label prédit: {pred_label}\n")

Echantillon de 10 prédictions incorrectes :

Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
Label réel: negative, Label prédit: positive

Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Label réel: negative, Label prédit: positive

Tweet: i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
Label réel: positive, Label prédit: negative

Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Label réel: negative, Label prédit: positive

Tweet: I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
Label réel: positive, Label prédit: negative

Tweet: wish I could feel no pain (8)  but it`s ok, at least they like Brazil!
Label réel: positive, Label prédit: n

---

## Étape 7 : Entraînement avec TfidfVectorizer

### Créer un modèle vectorizer avec la méthode TfidfVectorizer

In [14]:
# Initialisation du TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Appliquer le vectorizer sur les données d'entraînement
X_train_TF = tfidf_vectorizer.fit_transform(X_train)

# Transformer les données de test sans ré-entraîner
X_test_TF = tfidf_vectorizer.transform(X_test)

# Vérifier la forme des matrices
print(f"Dimensions X_train_TF : {X_train_TF.shape}")
print(f"Dimensions X_test_TF : {X_test_TF.shape}")

Dimensions X_train_TF : (12272, 15806)
Dimensions X_test_TF : (4091, 15806)


### Entraîner le modèle de régression logistique avec TfidfVectorizer

In [15]:
# Initialisation du modèle
model_tfidf = LogisticRegression()

# Entraîner le modèle
model_tfidf.fit(X_train_TF, y_train)

LogisticRegression()

### Évaluer la performance du modèle TfidfVectorizer

In [16]:
# Prédictions sur les ensembles
train_accuracy_tfidf = model_tfidf.score(X_train_TF, y_train)
test_accuracy_tfidf = model_tfidf.score(X_test_TF, y_test)

print(f"Train Accuracy (Tfidf): {train_accuracy_tfidf:.3f}")
print(f"Test Accuracy (Tfidf): {test_accuracy_tfidf:.3f}")

# Rapport de classification complet
y_pred_tfidf = model_tfidf.predict(X_test_TF)
print(classification_report(y_test, y_pred_tfidf))

Train Accuracy (Tfidf): 0.931
Test Accuracy (Tfidf): 0.873
              precision    recall  f1-score   support

    negative       0.86      0.87      0.87      1935
    positive       0.88      0.87      0.88      2156

    accuracy                           0.87      4091
   macro avg       0.87      0.87      0.87      4091
weighted avg       0.87      0.87      0.87      4091



---

## Étape 8 : Comparatif des modèles

### Comparer les performances des modèles CountVectorizer et TfidfVectorizer

In [18]:
print("Comparatif des modèles :\n")
print(f"Accuracy Train (CountVectorizer) : {train_accuracy:.3f}")
print(f"Accuracy Test (CountVectorizer) : {test_accuracy:.3f}\n")
print(f"Accuracy Train (TfidfVectorizer) : {train_accuracy_tfidf:.3f}")
print(f"Accuracy Test (TfidfVectorizer) : {test_accuracy_tfidf:.3f}")

Comparatif des modèles :

Accuracy Train (CountVectorizer) : 0.966
Accuracy Test (CountVectorizer) : 0.877

Accuracy Train (TfidfVectorizer) : 0.931
Accuracy Test (TfidfVectorizer) : 0.873


**Conclusion** :

- Si l'objectif principal est d'avoir une meilleure généralisation, le modèle avec TfidfVectorizer est plus approprié.
- Si la performance brute (accuracy) est prioritaire et que le léger overfitting est acceptable, le modèle avec CountVectorizer peut être choisi.